In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
#  importing library needed for image Generating
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# importing libraray needed for noramalization
from tensorflow.keras.layers import BatchNormalization


In [ ]:
# Load the data from the CSV file
data = pd.read_csv('glaucoma.csv')
data.iloc[0:309,0]  =  data.iloc[0:309,0].apply(lambda x : 'Im' + x[0:3] +'_ACRIMA'+x[3:])
data.iloc[309:,0] = data.iloc[309:,0].apply(lambda x : 'Im' + x[0:3] +'_g_ACRIMA'+x[3:])
data['Filename'] = data['Filename'].apply(lambda x: os.path.join('ACRIMA','Images', x))

In [ ]:
X = []
y = []
for i, row in data.iterrows():
    image_path = row['Filename']
    label = row['Glaucoma']
    image = load_img(image_path, target_size=(224, 224))
    image_array = img_to_array(image)
    X.append(image_array)
    y.append(label)

# adding the image column to my data
X=np.array(X)
X = X/255.0
y =np.array(y)

In [ ]:
# Split the data into train and validation sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)


In [ ]:

# Define the model architecture
Sequential_model = Sequential()
Sequential_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
Sequential_model.add(MaxPooling2D((2, 2)))
Sequential_model.add(Conv2D(64, (3, 3), activation='relu'))
Sequential_model.add(MaxPooling2D((2, 2)))
Sequential_model.add(Conv2D(128, (3, 3), activation='relu'))
Sequential_model.add(MaxPooling2D((2, 2)))
Sequential_model.add(Flatten())
Sequential_model.add(Dense(128, activation='relu'))
Sequential_model.add(Dropout(0.5))
Sequential_model.add(Dense(1, activation='sigmoid'))

In [ ]:

# Compile the model
Sequential_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the Sequential_model
Sequential_model_history = Sequential_model.fit(
    X_train, y_train,
    epochs=20,
    validation_data=(X_test, y_test),
    verbose=1)

# Evaluate the Sequential_model
loss, accuracy = Sequential_model.evaluate(X_test, y_test)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

In [ ]:
# The validation and accuracy graph of the sequentail trained model

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(Sequential_model_history.history['accuracy'])
plt.plot(Sequential_model_history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(Sequential_model_history.history['loss'])
plt.plot(Sequential_model_history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:


# Create an ImageDataGenerator object for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the data generator on the training data
datagen.fit(X_train)

# Define the model architecture
rotated_model = Sequential()
rotated_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
rotated_model.add(MaxPooling2D((2, 2)))
rotated_model.add(Conv2D(64, (3, 3), activation='relu'))
rotated_model.add(MaxPooling2D((2, 2)))
rotated_model.add(Conv2D(128, (3, 3), activation='relu'))
rotated_model.add(MaxPooling2D((2, 2)))
rotated_model.add(Flatten())
rotated_model.add(Dense(128, activation='relu'))
rotated_model.add(Dropout(0.5))
rotated_model.add(Dense(1, activation='sigmoid'))

# Compile the rotated_model
rotated_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the rotated_model using the augmented data
rotated_model_history = rotated_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=10,
    validation_data=(X_test, y_test),
    verbose=1
)
loss, accuracy = rotated_model.evaluate(X_test, y_test)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

In [ ]:
# The validation and accuracy graph of the sequentail trained model

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(rotated_model_history.history['accuracy'])
plt.plot(rotated_model_history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(rotated_model_history.history['loss'])
plt.plot(rotated_model_history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:


# Define the model architecture with regularization
normalized_model = Sequential()
normalized_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
normalized_model.add(BatchNormalization())
normalized_model.add(MaxPooling2D((2, 2)))
normalized_model.add(Conv2D(64, (3, 3), activation='relu'))
normalized_model.add(BatchNormalization())
normalized_model.add(MaxPooling2D((2, 2)))
normalized_model.add(Conv2D(128, (3, 3), activation='relu'))
normalized_model.add(BatchNormalization())
normalized_model.add(MaxPooling2D((2, 2)))
normalized_model.add(Flatten())
normalized_model.add(Dense(128, activation='relu'))
normalized_model.add(Dropout(0.5))
normalized_model.add(Dense(1, activation='sigmoid'))

# Compile the normalized_model
normalized_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the normalized_model
normalized_model_history = normalized_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_test, y_test),
    verbose=1
)
loss, accuracy = normalized_model.evaluate(X_test, y_test)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

In [ ]:
# The validation and accuracy graph of the sequentail trained model

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(normalized_model_history.history['accuracy'])
plt.plot(normalized_model_history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(normalized_model_history.history['loss'])
plt.plot(normalized_model_history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Plot training history
plt.plot(Sequential_model.history.history['accuracy'], label='Training Accuracy')
plt.plot(Sequential_model.history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

plt.plot(Sequential_model_history.history['loss'], label='Training Loss')
plt.plot(Sequential_model.history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Generate predictions for the test set
y_pred = (Sequential_model.predict(X_test) > 0.5).astype("int32")

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()